In [24]:
%%time
import time
time.sleep(2)
# Provides ways to work with large multidimensional arrays
import numpy as np 

# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline


# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

# import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

from IPython.display import display
# Setting display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

import os
import sys
import seaborn as sns
# from pathlib import Path
from dataset.us_equity_load import *
from utils.us_equity_utils import *

from factor.xq_finance import *
from dataset.us_equity_load import *
from factor.price import*
# %load_ext ipython_magic
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2
# %load_ext ipython_magic
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.instance().extension_manager.load_extension("autoreload")
# InteractiveShell.instance().extension_manager.load_extension("writefile")
import sys
print(sys.executable)


# import IPython
# print(IPython.__version__)
# from IPython.core.magic import register_line_cell_magic

# @register_line_cell_magic
# def writefile(line, cell=None):
#     with open(line, 'w') as f:
#         f.write(cell or '')


folder_name = './fator'
file_name = 'index.py'
# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
file_path = os.path.join('./', 'factor', 'index.py')


/opt/openbb/bin/python
CPU times: user 6.15 ms, sys: 2.66 ms, total: 8.81 ms
Wall time: 2.01 s


In [26]:
# %%writefile $file_path

from dataset.us_equity_load import *

class index:
  def __init__(self, sectors, start_date, end_date):
     self.sectors = sectors
     self.start_date = start_date
     self.end_date = end_date

  def calc(self):
      dict_index = {}
      for sector in self.sectors:
          symbols = us_dir0_load_csv(dir0 = 'symbol', filename= sector +'.csv')['symbol'].values
          data = us_equity_daily_data_load(symbols = symbols, start_date = self.start_date,
                                           end_date = self.end_date, trade_option = 'market_capital', 
                                           dir_option = 'xq')
          df = pd.DataFrame(data)
          df_sum = df.sum(axis=1)
          index = df_sum * 1000 /df_sum.iloc[0]
          dict_index[sector] = index

      return pd.DataFrame(dict_index)

  def ratio(self, days = 1):
     df = self.calc()
     return df.pct_change(days) * 100 

In [23]:
# %load_ext autoreload
# %autoreload 2
# %aimport factor.index

sector_name = "Semiconductors_Equipment"

# sector_name = "Automobiles"
# sector_name = "Internet_Retail"
# sector_name = "Electronic_Equipment_Instruments_Components"

start_date = '2024-01-29'
end_date = '2024-07-08'
# sectors = ["Semiconductors_Equipment", "Automobiles", "Internet_Retail", "Electronic_Equipment_Instruments_Components"]
sectors = ["Semiconductors_Equipment"]
index = index(sectors, start_date, end_date)
df = index.ratio()

print(df)

failed to load equity AVGOP
failed to load equity BRKS
failed to load equity CCMP
failed to load equity CREE
failed to load equity DSPG
failed to load equity MOSY
failed to load equity MXIM
failed to load equity NAL
failed to load equity NPTN
failed to load equity OIIM
lack of some trade date skip  RBCN
failed to load equity RESN
failed to load equity TYGOW
failed to load equity XLNX
                     Semiconductors_Equipment
date                                         
2024-01-29 05:00:00                       NaN
2024-01-30 05:00:00                 -0.009066
2024-01-31 05:00:00                 -0.016316
2024-02-01 05:00:00                  0.012295
2024-02-02 05:00:00                  0.023483
2024-02-05 05:00:00                  0.023130
2024-02-06 05:00:00                 -0.011122
2024-02-07 05:00:00                  0.023520
2024-02-08 05:00:00                  0.013011
2024-02-09 05:00:00                  0.022405
2024-02-12 05:00:00                 -0.004752
2024-02-13 05:0